In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
pip install keras-tuner


In [ ]:
import keras_tuner as kt

In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle-2.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle datasets list



In [ ]:
!kaggle datasets download -d farzadnekouei/trash-type-image-dataset

In [ ]:
!unzip trash-type-image-dataset.zip -d TrashType_Image_Dataset


In [ ]:
dataset_dir = "/content/TrashType_Image_Dataset/TrashType_Image_Dataset"


In [ ]:
image_size = (124, 124)
seed = 42

In [ ]:
# Split into train/validation dataset
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [ ]:
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='sparse',
    subset='training',
    seed=seed
)

val_generator = val_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='sparse',
    subset='validation',
    seed=seed
)

In [ ]:
num_classes = train_generator.num_classes

In [ ]:
# Model
def model_builder(hp):
    # Hyperparameters
    hp_learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    hp_optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])

    # Base model (frozen for tuning)
    base_model = EfficientNetV2B2(
        input_shape=(124, 124, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    # Build model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    # Optimizer selection
    if hp_optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
    else:
        optimizer = keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
# Hyperparameter Tuning
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='tuner_results',
    project_name='effnetv2_tuning'
)

In [ ]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [ ]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    callbacks=[stop_early]
)

best_hps = tuner.get_best_hyperparameters()[0]
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best optimizer: {best_hps.get('optimizer')}")


In [ ]:
# Train on the best model
model = tuner.hypermodel.build(best_hps)

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "best_model.h5", save_best_only=True, monitor='val_accuracy', mode='max'
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint_cb]
)

In [ ]:
# Fine-Tuning (Unfreeze Top Layers)
base_model = model.layers[0]
base_model.trainable = True

In [ ]:
# Freeze all but last 50 layers for fine-tuning
for layer in base_model.layers[:-50]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate') / 10),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[checkpoint_cb]
)


In [ ]:
# Load the best saved model
best_model = keras.models.load_model("best_model.h5")

In [ ]:
# Evaluate on validation set
loss, accuracy = best_model.evaluate(val_generator)
print(f"Final Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
loss, accuracy = best_model.evaluate(train_generator)
print(f"Final Validation Accuracy: {accuracy * 100:.2f}%")